In [1]:
import numpy as np
import pandas as pd
import catboost as cb
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import gmean
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [2]:
class Bayesian_optimization():
    def __init__(self, classifier, params, params_search,
                 x_train, y_train, x_valid, y_valid,
                 categorical=None,
                 metric=roc_auc_score, evals=True, 
                 method='maximize', random_state=42,
                 init_points=3, n_iter=5, acq='ei',
                 early_stopping_rounds=50, verbose=200,
                 int_params=[]):
        self.classifier = classifier
        self.params = params
        self.params_search = params_search
        self.x_train = x_train
        self.y_train = y_train
        self.x_valid = x_valid
        self.y_valid = y_valid
        self.categorical = categorical
        self.metric = metric
        self.evals = [(x_train, y_train), (x_valid, y_valid)] \
                        if evals else None
        self.method = method
        self.random_state = random_state
        self.init_points = init_points
        self.n_iter = n_iter
        self.acq = acq
        self.early_stopping_rounds = early_stopping_rounds
        self.verbose = verbose
        self.int_params = int_params
        self.model = None
    
    def _normalize_params(self, params: dict):
        new_params = params.copy()
        for int_param in self.int_params:
            if int_param in params.keys():
                new_params[int_param] = int(params[int_param])
        return new_params
    
    def _optimize_params(self, **kwargs):
        params = {**self.params,**kwargs}
        params = self._normalize_params(params)
        
        model = self.classifier(**params)
        model.fit(
            self.x_train, self.y_train, 
#             categorical=self.categorical,
            eval_set=self.evals,
            early_stopping_rounds=self.early_stopping_rounds,
            verbose=self.verbose
        )
        return self.metric(self.y_valid, model.predict(self.x_valid))
    
    def _optimization(self):
        optimizer = BayesianOptimization(
            self._optimize_params,
            pbounds=self.params_search,
            random_state=self.random_state
        )
    #     if method == 'maximize':
        optimizer.maximize(init_points=self.init_points, 
                           n_iter=self.n_iter, 
                           acq=self.acq)
        return optimizer.max
    
    def optimize_it(self):
        optimal = self._optimization()
        optimal_params = optimal['params']
        optimal_params = self._normalize_params(optimal_params)
        
        self.model = self.classifier(**optimal_params)
        self.model.fit(self.x_train, self.y_train, 
                       eval_set=self.evals,
                       early_stopping_rounds=self.early_stopping_rounds,
                       verbose=self.verbose)

        return optimal_params, optimal

In [3]:
train = pd.read_pickle('./prepared_train_adv075.pkl')
valid = pd.read_pickle('./prepared_valid_adv075.pkl')
test = pd.read_pickle('./prepared_test_adv075.pkl')
target_name = 'target'

In [4]:
y_train = train[target_name]
y_valid = valid[target_name]
x_train = train.drop(target_name, axis=1)
x_valid = valid.drop(target_name, axis=1)

In [5]:
categorical = [
    'name_contract_type',
    'gender',
    'education_level_x',
    'family_status',
    'education_level_family_status',
    'education_level_y'
]
numerical = list(set(train.columns) - set(categorical))

In [6]:
model_search = {
#     'xgb': {
#         'classifier': xgb.XGBClassifier,
#         'params': {
#             "booster": "gblinear",
#             "objective": "binary:logistic",
#             "eval_metric": "auc",
#             "n_estimators": 10000,
#             "seed": 42,
#         },
#         'params_search': {
#             "max_depth": (3, 7),
#             "subsample": (0.3, 0.9),
#             "colsample_bytree": (0.3, 0.9),
#             "min_child_weights": (300, 700),
#             "alpha": (1, 20),
#             "lambda": (10, 100), 
#             "gamma": (1, 20), 
#             "eta": (1e-2, 0.1)
#         }
#     },
    'lgb1': {
        'classifier': lgb.LGBMClassifier,
        'params': {
            'metric':"auc", 
            'boosting_type': 'gbdt',
            'n_estimators': 10000,
            'random_state': 42,
        },
        'params_search': {
            "max_depth": (4, 8),
            "bagging_fraction": (0.2, 0.6),
            "feature_fraction": (0.2, 0.5),
            "min_data_in_leaf": (300, 550),
            "lambda_l1": (8, 13),
            "labmda_l2": (70, 75), 
            "learning_rate": (0.05, 0.2)
        }
    },
    'lgb2': {
        'classifier': lgb.LGBMClassifier,
        'params': {
            'metric':"auc", 
            'boosting_type': 'gbdt',
            'n_estimators': 10000,
            'random_state': 1,
        },
        'params_search': {
            "max_depth": (4, 8),
            "bagging_fraction": (0.2, 0.6),
            "feature_fraction": (0.2, 0.5),
            "min_data_in_leaf": (300, 550),
            "lambda_l1": (8, 13),
            "labmda_l2": (70, 75), 
            "learning_rate": (0.05, 0.2)
        }
    },
    'lgb3': {
        'classifier': lgb.LGBMClassifier,
        'params': {
            'metric':"auc", 
            'boosting_type': 'gbdt',
            'n_estimators': 10000,
            'random_state': 10,
        },
        'params_search': {
            "max_depth": (4, 8),
            "bagging_fraction": (0.2, 0.6),
            "feature_fraction": (0.2, 0.5),
            "min_data_in_leaf": (300, 550),
            "lambda_l1": (8, 13),
            "labmda_l2": (70, 75), 
            "learning_rate": (0.05, 0.2)
        }
    },
    'cb1': {
        'classifier': cb.CatBoostClassifier,
        'params': {
            'n_estimators': 10000,
            'loss_function': 'Logloss',
            'eval_metric': 'AUC',
            'task_type': 'CPU',
            'random_seed': 42,
        },
        'params_search': {
            "max_depth": (4, 8),
            "subsample": (0.4, 0.9),
            "colsample_bylevel": (0.4, 0.9),
            "min_data_in_leaf": (300, 550),
            "reg_lambda": (1, 10),
            "learning_rate": (0.04, 0.1)
        }
    },
    'cb2': {
        'classifier': cb.CatBoostClassifier,
        'params': {
            'n_estimators': 10000,
            'loss_function': 'Logloss',
            'eval_metric': 'AUC',
            'task_type': 'CPU',
            'random_seed': 1,
        },
        'params_search': {
            "max_depth": (4, 8),
            "subsample": (0.4, 0.9),
            "colsample_bylevel": (0.4, 0.9),
            "min_data_in_leaf": (300, 550),
            "reg_lambda": (1, 10),
            "learning_rate": (0.04, 0.1)
        }
    },
    'cb3': {
        'classifier': cb.CatBoostClassifier,
        'params': {
            'n_estimators': 10000,
            'loss_function': 'Logloss',
            'eval_metric': 'AUC',
            'task_type': 'CPU',
            'random_seed': 10,
        },
        'params_search': {
            "max_depth": (4, 8),
            "subsample": (0.4, 0.9),
            "colsample_bylevel": (0.4, 0.9),
            "min_data_in_leaf": (300, 550),
            "reg_lambda": (1, 10),
            "learning_rate": (0.04, 0.1)
        }
    },
}

In [7]:
%%time
models = dict()
LB_preds = pd.DataFrame()

for key, value in model_search.items():
    model_name = key
    classifier = value['classifier']
    params = value['params']
    params_search = value['params_search']
    categorical = categorical
    
    print(f'{model_name}\n{"*"*70}\n{"*"*70}')

    optimizator = Bayesian_optimization(
                    classifier, params, params_search,
                    x_train, y_train, x_valid, y_valid,
#                     categorical=categorical,
                    metric=roc_auc_score, evals=True, 
                    method='maximize', random_state=42,
                    init_points=4, n_iter=120, acq='ei',
                    early_stopping_rounds=50, verbose=200,
                    int_params=['max_depth', 
                                'min_child_weights', 
                                'min_data_in_leaf']
                  )
    optimal_params, optimal = optimizator.optimize_it()
    
    model = optimizator.model
    model.fit(x_train, y_train)
#     print(f'\n{model_name}\ntest metric: \
#         {roc_auc_score(y_test, model.predict(x_test))}\n')
    
    models[model_name] = {
        'model': model,
        'y_valid_pred': model.predict(x_valid),
#         'y_test_pred': model.predict(x_test),
        'optimal_params': optimal_params,
        'optimal': optimal
    }
    
    LB_preds[model_name] = model.predict_proba(test)[:, 1]

lgb1
**********************************************************************
**********************************************************************
|   iter    |  target   | baggin... | featur... | labmda_l2 | lambda_l1 | learni... | max_depth | min_da... |
-------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] Unknown parameter: labmda_l2
[LightGBM] [Warning] feature_fraction is set=0.4852142919229748, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4852142919229748
[LightGBM] [Warning] min_data_in_leaf is set=314, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=314
[LightGBM] [Warning] lambda_l1 is set=10.993292420985183, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10.993292420985183
[LightGBM] [Warning] bagging_fraction is set=0.349816047538945, subsample=1.0 will be ignored. Current value: bagging_fraction=0.349816047538945
Training until validation s

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[60]	training's auc: 0.783928	valid_1's auc: 0.72236
|  9        |  0.5049   |  0.4224   |  0.4711   |  73.31    |  9.153    |  0.1949   |  5.585    |  306.9    |
[LightGBM] [Warning] Unknown parameter: labmda_l2
[LightGBM] [Warning] feature_fraction is set=0.25782386250651634, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25782386250651634
[LightGBM] [Warning] min_data_in_leaf is set=404, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=404
[LightGBM] [Warning] lambda_l1 is set=11.254857520151567, reg_alpha=0.0 will be ignored. Current value: lambda_l1=11.254857520151567
[LightGBM] [Warning] bagging_fraction is set=0.5236526443566715, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5236526443566715
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[145]	training's auc: 0.778554	vali

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[100]	training's auc: 0.797226	valid_1's auc: 0.72351
|  18       |  0.505    |  0.3977   |  0.4422   |  72.75    |  12.76    |  0.1207   |  7.762    |  378.7    |
[LightGBM] [Warning] Unknown parameter: labmda_l2
[LightGBM] [Warning] feature_fraction is set=0.3015398459856031, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3015398459856031
[LightGBM] [Warning] min_data_in_leaf is set=486, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=486
[LightGBM] [Warning] lambda_l1 is set=11.279790725900666, reg_alpha=0.0 will be ignored. Current value: lambda_l1=11.279790725900666
[LightGBM] [Warning] bagging_fraction is set=0.4985544757263884, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4985544757263884
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[128]	training's auc: 0.771808	valid

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[35]	training's auc: 0.775182	valid_1's auc: 0.720345
|  27       |  0.5039   |  0.2251   |  0.4953   |  71.15    |  8.366    |  0.1987   |  7.511    |  521.4    |
[LightGBM] [Warning] Unknown parameter: labmda_l2
[LightGBM] [Warning] feature_fraction is set=0.35787180941579133, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.35787180941579133
[LightGBM] [Warning] min_data_in_leaf is set=307, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=307
[LightGBM] [Warning] lambda_l1 is set=8.057212852352382, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.057212852352382
[LightGBM] [Warning] bagging_fraction is set=0.43043424893565446, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43043424893565446
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[84]	training's auc: 0.792405	vali

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[128]	training's auc: 0.80455	valid_1's auc: 0.722616
|  36       |  0.5044   |  0.545    |  0.4121   |  70.8     |  11.71    |  0.1489   |  5.294    |  308.8    |
[LightGBM] [Warning] Unknown parameter: labmda_l2
[LightGBM] [Warning] feature_fraction is set=0.36333399441175096, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.36333399441175096
[LightGBM] [Warning] min_data_in_leaf is set=541, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=541
[LightGBM] [Warning] lambda_l1 is set=11.025052579105507, reg_alpha=0.0 will be ignored. Current value: lambda_l1=11.025052579105507
[LightGBM] [Warning] bagging_fraction is set=0.20958708278386431, subsample=1.0 will be ignored. Current value: bagging_fraction=0.20958708278386431
Training until validation scores don't improve for 50 rounds
[200]	training's auc: 0.766858	valid_1's auc: 0.725473
Early stoppi

...

219:	learn: 0.2496418	total: 5.94s	remaining: 21.1s
220:	learn: 0.2496169	total: 5.97s	remaining: 21.1s
221:	learn: 0.2495752	total: 6s	remaining: 21s
222:	learn: 0.2495267	total: 6.04s	remaining: 21s
223:	learn: 0.2494588	total: 6.06s	remaining: 21s
224:	learn: 0.2494005	total: 6.1s	remaining: 21s
225:	learn: 0.2493523	total: 6.12s	remaining: 21s
226:	learn: 0.2493102	total: 6.16s	remaining: 21s
227:	learn: 0.2492485	total: 6.18s	remaining: 20.9s
228:	learn: 0.2491945	total: 6.22s	remaining: 20.9s
229:	learn: 0.2491798	total: 6.24s	remaining: 20.9s
230:	learn: 0.2491168	total: 6.29s	remaining: 20.9s
231:	learn: 0.2490735	total: 6.31s	remaining: 20.9s
232:	learn: 0.2490560	total: 6.36s	remaining: 20.9s
233:	learn: 0.2489958	total: 6.38s	remaining: 20.9s
234:	learn: 0.2489674	total: 6.42s	remaining: 20.9s
235:	learn: 0.2489119	total: 6.44s	remaining: 20.9s
236:	learn: 0.2488649	total: 6.48s	remaining: 20.9s
237:	learn: 0.2488226	total: 6.51s	remaining: 20.8s
238:	learn: 0.2487952	total:

380:	learn: 0.2422459	total: 10.9s	remaining: 17.6s
381:	learn: 0.2422036	total: 10.9s	remaining: 17.6s
382:	learn: 0.2421568	total: 10.9s	remaining: 17.6s
383:	learn: 0.2421056	total: 10.9s	remaining: 17.5s
384:	learn: 0.2420644	total: 11s	remaining: 17.5s
385:	learn: 0.2420334	total: 11s	remaining: 17.5s
386:	learn: 0.2419839	total: 11s	remaining: 17.5s
387:	learn: 0.2419645	total: 11.1s	remaining: 17.4s
388:	learn: 0.2419468	total: 11.1s	remaining: 17.4s
389:	learn: 0.2418901	total: 11.1s	remaining: 17.4s
390:	learn: 0.2418726	total: 11.1s	remaining: 17.4s
391:	learn: 0.2418499	total: 11.2s	remaining: 17.3s
392:	learn: 0.2418086	total: 11.2s	remaining: 17.3s
393:	learn: 0.2417536	total: 11.2s	remaining: 17.3s
394:	learn: 0.2417497	total: 11.3s	remaining: 17.2s
395:	learn: 0.2417285	total: 11.3s	remaining: 17.2s
396:	learn: 0.2417114	total: 11.3s	remaining: 17.2s
397:	learn: 0.2416735	total: 11.3s	remaining: 17.2s
398:	learn: 0.2416233	total: 11.4s	remaining: 17.1s
399:	learn: 0.2415

539:	learn: 0.2361193	total: 15.6s	remaining: 13.3s
540:	learn: 0.2360585	total: 15.6s	remaining: 13.3s
541:	learn: 0.2360280	total: 15.6s	remaining: 13.2s
542:	learn: 0.2359676	total: 15.7s	remaining: 13.2s
543:	learn: 0.2359197	total: 15.7s	remaining: 13.2s
544:	learn: 0.2358617	total: 15.7s	remaining: 13.1s
545:	learn: 0.2357987	total: 15.8s	remaining: 13.1s
546:	learn: 0.2357599	total: 15.8s	remaining: 13.1s
547:	learn: 0.2357182	total: 15.8s	remaining: 13.1s
548:	learn: 0.2356578	total: 15.9s	remaining: 13s
549:	learn: 0.2356154	total: 15.9s	remaining: 13s
550:	learn: 0.2355584	total: 15.9s	remaining: 13s
551:	learn: 0.2355086	total: 15.9s	remaining: 12.9s
552:	learn: 0.2354776	total: 16s	remaining: 12.9s
553:	learn: 0.2354427	total: 16s	remaining: 12.9s
554:	learn: 0.2354026	total: 16s	remaining: 12.8s
555:	learn: 0.2353613	total: 16s	remaining: 12.8s
556:	learn: 0.2353326	total: 16.1s	remaining: 12.8s
557:	learn: 0.2352990	total: 16.1s	remaining: 12.8s
558:	learn: 0.2352530	tota

702:	learn: 0.2298643	total: 20.4s	remaining: 8.63s
703:	learn: 0.2298333	total: 20.5s	remaining: 8.6s
704:	learn: 0.2297856	total: 20.5s	remaining: 8.58s
705:	learn: 0.2297612	total: 20.5s	remaining: 8.55s
706:	learn: 0.2297366	total: 20.6s	remaining: 8.52s
707:	learn: 0.2297305	total: 20.6s	remaining: 8.49s
708:	learn: 0.2296936	total: 20.6s	remaining: 8.46s
709:	learn: 0.2296554	total: 20.6s	remaining: 8.43s
710:	learn: 0.2296009	total: 20.7s	remaining: 8.41s
711:	learn: 0.2295956	total: 20.7s	remaining: 8.38s
712:	learn: 0.2295553	total: 20.7s	remaining: 8.35s
713:	learn: 0.2295333	total: 20.8s	remaining: 8.32s
714:	learn: 0.2295024	total: 20.8s	remaining: 8.29s
715:	learn: 0.2294734	total: 20.8s	remaining: 8.26s
716:	learn: 0.2294154	total: 20.9s	remaining: 8.24s
717:	learn: 0.2293897	total: 20.9s	remaining: 8.21s
718:	learn: 0.2293625	total: 20.9s	remaining: 8.18s
719:	learn: 0.2293273	total: 21s	remaining: 8.15s
720:	learn: 0.2292875	total: 21s	remaining: 8.12s
721:	learn: 0.229

862:	learn: 0.2243141	total: 25.3s	remaining: 4.01s
863:	learn: 0.2242757	total: 25.3s	remaining: 3.98s
864:	learn: 0.2242366	total: 25.3s	remaining: 3.95s
865:	learn: 0.2242022	total: 25.4s	remaining: 3.92s
866:	learn: 0.2241735	total: 25.4s	remaining: 3.9s
867:	learn: 0.2241574	total: 25.4s	remaining: 3.87s
868:	learn: 0.2241341	total: 25.5s	remaining: 3.84s
869:	learn: 0.2240949	total: 25.5s	remaining: 3.81s
870:	learn: 0.2240563	total: 25.5s	remaining: 3.78s
871:	learn: 0.2240427	total: 25.6s	remaining: 3.75s
872:	learn: 0.2240057	total: 25.6s	remaining: 3.72s
873:	learn: 0.2239665	total: 25.6s	remaining: 3.69s
874:	learn: 0.2239401	total: 25.7s	remaining: 3.67s
875:	learn: 0.2239002	total: 25.7s	remaining: 3.63s
876:	learn: 0.2238664	total: 25.7s	remaining: 3.61s
877:	learn: 0.2238449	total: 25.7s	remaining: 3.58s
878:	learn: 0.2238065	total: 25.8s	remaining: 3.55s
879:	learn: 0.2237551	total: 25.8s	remaining: 3.52s
880:	learn: 0.2237205	total: 25.8s	remaining: 3.49s
881:	learn: 0

In [8]:
for key, val in models.items():
    print(f'\n{key}\n{val["optimal_params"]}')


lgb1
{'bagging_fraction': 0.4245984543805995, 'feature_fraction': 0.37016629100281384, 'labmda_l2': 72.22012023183261, 'lambda_l1': 8.524774386593757, 'learning_rate': 0.18557454434490395, 'max_depth': 6, 'min_data_in_leaf': 490}

lgb2
{'bagging_fraction': 0.5770777845234509, 'feature_fraction': 0.39347928750752537, 'labmda_l2': 70.05818243715893, 'lambda_l1': 11.693280641236115, 'learning_rate': 0.19124988844449842, 'max_depth': 6, 'min_data_in_leaf': 442}

lgb3
{'bagging_fraction': 0.30985512418982875, 'feature_fraction': 0.21722505303850836, 'labmda_l2': 73.64542723691177, 'lambda_l1': 9.105921537661837, 'learning_rate': 0.14312101677883687, 'max_depth': 6, 'min_data_in_leaf': 490}

cb1
{'colsample_bylevel': 0.7457106420581108, 'learning_rate': 0.06408588057786521, 'max_depth': 5, 'min_data_in_leaf': 320, 'reg_lambda': 3.619091358582655, 'subsample': 0.6524339996208841}

cb2
{'colsample_bylevel': 0.8142548545164984, 'learning_rate': 0.08214392931532896, 'max_depth': 4, 'min_data_in

In [9]:
LB_preds.head(3)

,lgb1,lgb2,lgb3,cb1,cb2,cb3
0,0.102037,0.085365,0.105698,0.106897,0.090976,0.106913
1,0.199770,0.298285,0.291800,0.248918,0.261927,0.256192
2,0.164599,0.166013,0.136082,0.201909,0.204832,0.186023


In [10]:
# y_pred = gmean(LB_preds, axis=1)
y_pred = LB_preds.mean(axis=1)
# y_pred = y_pred * 4
y_pred = y_pred.clip(0, 1)

In [11]:
submit = pd.read_csv('./data/sample_submit.csv')
submit['TARGET'] = y_pred
submit.to_csv('./optimazed_preds.csv', index=False, encoding='utf-8', sep=',')
submit.head()

,APPLICATION_NUMBER,TARGET
0,123724268,0.099648
1,123456549,0.259482
2,123428178,0.176576
3,123619984,0.074665
4,123671104,0.016473


In [12]:
submit[submit['TARGET'] > 0.5].shape

(368, 2)